In [ ]:
from bs4 import BeautifulSoup
import base64

In [52]:
import pytesseract
from PIL import Image, ImageFilter
import cv2
from collections import Counter
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
import time
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [78]:
def preprocess_captcha(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding (binarization)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    
    # Optionally, apply noise removal (median filter)
    denoised = cv2.medianBlur(binary, 3)
    
    # Save the preprocessed image for debugging
    preprocessed_path = "preprocessed_captcha.png"
    cv2.imwrite(preprocessed_path, denoised)
    
    return preprocessed_path

def read_captcha(captcha_path):
    # Preprocess the captcha image
    preprocessed_path = preprocess_captcha(captcha_path)
    
    # Read the preprocessed image with pytesseract
    captcha_text = pytesseract.image_to_string(Image.open(preprocessed_path), config="--psm 7")
    
    return captcha_text.strip()

def run_multiple_captcha_attempts(captcha_patch,attempts=10):
    results = []
    
    for i in range(attempts):
        # print(f"Attempt {i + 1}...")
        # Download captcha
        # if download_captcha(captcha_url):  # This correctly downloads the captcha image
            # Read and process the captcha
            captcha_text = read_captcha(captcha_patch)
            results.append(captcha_text)
            # print(f"Extracted CAPTCHA (Attempt {i + 1}): {captcha_text}")
    print(results)
    # Determine the most repeated result
    most_common = Counter(results).most_common(1)
    if most_common:
        print(f"\nMost repeated CAPTCHA result: {most_common[0][0].replace(" ", "")}")
    
    return most_common[0][0].replace(" ", "")


In [96]:
# Mở trình duyệt để nhập captcha
print("🔓 Mở trình duyệt để nhập captcha...")
driver = webdriver.Chrome()
# driver.get("https://trade.ec.europa.eu/access-to-markets/en/search?product=0101&origin=VN&destination=AT")  # Thay bằng URL cần mở
driver.get("https://trade.ec.europa.eu/access-to-markets/en/captcha?redirectTo=%2Fen%2Fsearch%3Fproduct%3D0101%26origin%3DVN%26destination%3DAT")
print("⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt.")

# Đo thời gian bắt đầu
start_captcha_time = time.time()
wait_time = 1800  # 30 phút 

# # Chờ người dùng tự đóng trình duyệt, nếu quá 10 phút thì tự động đóng
# while True:
#     try:
#         if time.time() - start_captcha_time > wait_time:
#             print("⏳ Quá 10 phút, tự động đóng trình duyệt.")
#             driver.quit()
#             break  # Thoát khỏi vòng lặp

#         # Kiểm tra trình duyệt còn mở không
#         driver.title  
        
#         # Kiểm tra trang có bị treo không
#         if driver.execute_script("return document.readyState") != "complete":
#             print("⚠️ Trang web có thể bị treo, chờ thêm...")
        
#     except WebDriverException:
#         print("✅ Trình duyệt đã được đóng.")
#         break  # Nếu trình duyệt bị đóng, thoát vòng lặp
    
#     time.sleep(5)  # Tránh vòng lặp chạy quá nhanh, gây tải CPU

🔓 Mở trình duyệt để nhập captcha...
⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt.


In [95]:
try:
    time.sleep(2)
    label = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "wt-captcha--text-challenge"))
    )
    label_html = label.get_attribute("outerHTML")
    # print("Nội dung của label:", label_html)
except Exception as e:
    print("Không tìm thấy thẻ label:", e)
soup = BeautifulSoup(label_html, 'html.parser')
base64_string = soup.find('img' , alt="First number")['src']
base64_string = base64_string.replace('data:image/png;base64,','')
image_data = base64.b64decode(base64_string)
with open("img1.png", "wb") as file:
    file.write(image_data)
base64_string2 = soup.find('img' , alt="Second number")['src']
base64_string2 = base64_string2.replace('data:image/png;base64,','')
image_data2 = base64.b64decode(base64_string2)
with open("img2.png", "wb") as file:
    file.write(image_data2)
img1_txt  =run_multiple_captcha_attempts("img1.png")
img2_txt  =run_multiple_captcha_attempts("img2.png")
# Thay thế thẻ <img> có alt="First number"
img1_tag = soup.find("img", alt="First number")
if img1_tag:
    img1_tag.replace_with(img1_txt)

# Thay thế thẻ <img> có alt="second number"
img2_tag = soup.find("img", alt="Second number")
if img2_tag:
    img2_tag.replace_with(img2_txt)
captcha_text = soup.text.replace('\n','').strip()

print(captcha_text)

['. (06ty-tive', '. (06ty-tive', '. (06ty-tive', '. (06ty-tive', '. (06ty-tive', '. (06ty-tive', '. (06ty-tive', '. (06ty-tive', '. (06ty-tive', '. (06ty-tive']

Most repeated CAPTCHA result: .(06ty-tive
['Forty-even', 'Forty-even', 'Forty-even', 'Forty-even', 'Forty-even', 'Forty-even', 'Forty-even', 'Forty-even', 'Forty-even', 'Forty-even']

Most repeated CAPTCHA result: Forty-even
Select the number .(06ty-tive or Forty-even by moving the slider.


In [89]:
try:
    time.sleep(2)
    error_element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "info-error"))
    )
    error_text = error_element.text
    print(error_text)
    if error_element:
        pass:
except: 
    pass

The answer provided was wrong, please try again


In [98]:
from selenium.webdriver.common.action_chains import ActionChains

In [113]:
# Tìm phần tử slider (Thay XPATH bằng selector của slider)
slider = driver.find_element("xpath", '//*[@id="__tab1__htspk63si4h"]/div/div/div[2]')

# Sử dụng ActionChains để kéo slider
action = ActionChains(driver)
action.click_and_hold(slider).move_by_offset(23, 7).release().perform()

#  đọc đoạn text sau khi xử lý ảnh captcha để trả về 1 chữ số
#  kéo thanh chain đến số đã chọn tìm  mỗi số là bao nhiêu pixel
#  chạy vòng lặp kiểm tra error_element